<a href="https://colab.research.google.com/github/Ghonem22/Convert-h5-model-to-tflite-with-inference/blob/main/Convert_h5_to_tflite_with_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf 
import numpy as np


# monunt drive to get the model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#load the model

In [25]:
# Loading Tensorflow model 
model = tf.keras.models.load_model('/content/currency_model.h5')

# convert model into tflite with reduceing its size

In [26]:
# Select mode of optimization
mode = "Speed" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [27]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Set the optimzations
converter.optimizations = [optimization]
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp3n7_mjj9/assets


INFO:tensorflow:Assets written to: /tmp/tmp3n7_mjj9/assets


## save the result

In [6]:
# Save the model.
with open('currency.tflite', 'wb') as f:
  f.write(tflite_model)

## build image clasification inference using tflite

### Load and prepare the model

In [8]:
import cv2
import tensorflow as tf
import numpy as np
classes = ['100',  '100', '10',  '10',  '200',  '200',  '20', '20',  '50',  '50',  '5', '5']

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter("/content/currency.tflite")
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]


In [9]:
def pred_currency(path):
    img=cv2.imread(path)
    dim=(400,400)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    img = tf.cast(img, tf.float32)
    img = img / 255
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)[0]

    if max(predictions) < 0.45:
        print("Please take another picture")
    else:    
        return classes[list(predictions).index(max(predictions))]




In [10]:
#predection
pred_currency("/content/currency.jpg")     

'100'

### move tflite model to drive for further using

In [28]:
import shutil
shutil.copy("/content/currency.tflite","/content/drive/MyDrive/Projects/IVA")

'/content/drive/MyDrive/Projects/IVA/currency.tflite'